## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



They use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [1]:
using Plots
using IJulia

In [2]:
using TensorFlow
using Distributions
using StatsBase

In [3]:
using DensityEstimationML
function only(itr)
    state = start(itr)
    val,state = next(itr, state)
    @assert(done(itr,state))
    return val
end

only (generic function with 1 method)

In [4]:
"""
Function returning a function that will display a running plot.
WARNING: Introducting or removing any variables is not supported.
And will silently error.
"""
function running_plot()
    epochs = Int[]
    record = Dict()
    function inner(epoch, vars::Associative)
        for (var, values) in vars
            value = only(values) #Incase it was an array
            past = get!(record, var) do
                typeof(value)[]
            end
            push!(past, value)
        end
        push!(epochs, epoch)
        
        IJulia.clear_output(true)
        plot(epochs, hcat(values(record)...); label=hcat(keys(vars)...), layout=length(vars)) |> IJulia.display       
    end
end


running_plot

In [5]:
function demonstration_plot(est, dataset, data, args...; kwargs...)
    X = minimum(approximate_support(dataset)) : 0.01 : maximum(approximate_support(dataset))
    @show typeof(data)
    println("True loglikelihood      = $(loglikelihood(dataset, data))")
    println("Estimated loglikelihood = $(loglikelihood(est, data))")
    plot([X], [pdf(est,X), data],
        #xlims= approximate_support(dataset),
        xlims= (first(X), last(X)),
        seriestype = [:path :histogram],
        layout=(2,1),
        legend=false,
        nbins=[1  length(data)÷10],
        args...; kwargs...
    )
end

demonstration_plot (generic function with 1 method)

In [6]:
function demo(dataset, layers, epochs=20_000; max_conditioning_epochs=2000)
    data = original_sample(dataset)
    @show loglikelihood(dataset, data)
    est = NeuralDensityEstimator(layers, approximate_support(dataset))

    condition!(est; max_epochs = max_conditioning_epochs)
    println("Conditioning Done")
    Plots.gr()
    fit!(est, data; epochs=epochs, callback=running_plot())
    println("Fitting Done")
    plotly()
    demonstration_plot(est, dataset, data) |> IJulia.display
    
    est    
end

demo (generic function with 2 methods)

In [7]:
est=demo(GenerateDatasets.Likas1(), [64, 64, 256], 1_000)

LoadError: [91mMethodError: no method matching setcharheight(::Float64)
The applicable method may be too new: running in world age 24188, while current world is 24195.[0m
Closest candidates are:
  setcharheight(::Real) at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/GR/src/GR.jl:1010 (method too new to be called from this world context.)[39m

In [8]:
loglikelihood(GenerateDatasets.Likas1(), [2.0])

-2.0794415416798357

In [9]:
GenerateDatasets.Likas1()

DensityEstimationML.GenerateDatasets.Likas1()

In [10]:
sort(component_weights(est))

LoadError: [91mUndefVarError: est not defined[39m

In [11]:
demo(GenerateDatasets.Likas2(), [64,64], 20_000)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 10 
 
 
 20 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.6177,175.813 56.7642,157.972 57.9107,90.3604 59.0572,11.811 60.2038,20.9728 61.3503,39.0803 62.4968,48.7998 63.6433,53.5195 64.7898,56.7424 65.9364,59.5442 
 67.0829,62.2701 68.2294,64.865 69.3759,67.4501 70.5225,70.0967 71.669,72.6509 72.8155,74.9688 73.962,77.0484 75.1085,78.984 76.2551,80.8907 77.4016,82.8609 
 78.5481,84.9445 79.6946,87.1438 80.8412,89.4208 81.9877,91.7138 83.1342,93.9549 84.2807,96.0961 85.4273,98.1525 86.5738,100.175 87.7203,102.237 88.8668,104.311 
 90.0133,106.228 91.1599,108.097 92.3064,109.974 93.4529,111.836 94.5994,113.649 95.746,115.392 96.8925,117.053 98.039,118.624 99.1855,120.104 100.332,121.496 
 101.479,122.803 102.625,124.029 103.772,125.174 104.918,126.25 106.065,127.232 107.211,128.131 108.358,128.844 109.504,129.545 110.651,130.416 111.797,131.305 
 112.944,132.237 114.09,133.229 115.237,134.267 116.383,135.337 117.53,136.567 118.676,137.894 119.823,139.183 120.969,140.347 122.116,141.392 123.263,142.311 
 124.409,143.131 125.556,143.848 126.702,144.529 127.849,145.257 128.995,146.007 130.142,146.771 131.288,147.511 132.435,148.205 133.581,148.84 134.728,149.414 
 135.874,149.956 137.021,150.463 138.167,150.961 139.314,151.459 140.46,151.968 141.607,152.491 142.753,153.031 143.9,153.586 145.046,154.146 146.193,154.678 
 147.339,155.17 148.486,155.614 149.633,156.021 150.779,156.397 151.926,156.753 153.072,157.091 154.219,157.419 155.365,157.734 156.512,158.037 157.658,158.325 
 158.805,158.6 159.951,158.863 161.098,159.115 162.244,159.361 163.391,159.599 164.537,159.833 165.684,160.067 166.83,160.298 167.977,160.53 169.123,160.763 
 170.27,161 171.416,161.235 172.563,161.469 173.71,161.703 174.856,161.935 176.003,162.172 177.149,162.421 178.296,162.685 179.442,162.962 180.589,163.248 
 181.735,163.538 182.882,163.829 184.028,164.113 185.175,164.396 186.321,164.67 187.468,164.933 188.614,165.195 189.761,165.438 190.907,165.672 192.054,165.897 
 193.2,166.113 194.347,166.313 195.493,166.501 196.64,166.674 197.786,166.831 198.933,166.977 200.08,167.113 201.226,167.238 202.373,167.353 203.519,167.463 
 204.666,167.57 205.812,167.666 206.959,167.767 208.105,167.857 209.252,167.95 210.398,168.038 211.545,168.118 212.691,168.195 213.838,168.254 214.984,168.305 
 216.131,168.334 217.277,168.364 218.424,168.389 219.57,168.437 220.717,168.488 221.863,168.539 223.01,168.584 224.157,168.624 225.303,168.652 226.45,168.674 
 227.596,168.691 228.743,168.71 229.889,168.749 231.036,168.805 232.182,168.874 233.329,168.951 234.475,169.04 235.622,169.139 236.768,169.244 237.915,169.361 
 239.061,169.482 240.208,169.604 241.354,169.723 242.501,169.84 243.647,169.964 244.794,170.096 245.94,170.238 247.087,170.345 248.233,170.212 249.38,170.589 
 250.527,173.059 251.673,173.485 252.82,173.153 253.966,172.778 255.113,172.536 256.259,172.445 257.406,172.434 258.552,172.451 259.699,172.478 260.845,172.512 
 261.992,172.547 263.138,172.582 264.285,172.617 265.431,172.654 266.578,172.693 267.724,172.729 268.871,172.765 270.017,172.802 271.164,172.84 272.31,172.876 
 273.457,172.912 274.604,172.946 275.75,172.985 276.897,173.019 278.043,173.052 279.19,173.085 280.336,173.116 281.483,173.146 282.629,173.175 283.776,173.204 
 
 "/>
 
 
 
 
 ysmax 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 10000 
 
 
 15000 
 
 
 20000 
 
<polyline clip-path="url(#clip03)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 352.157,11.811 353.303,109.406 354.45,148.447 355.596,157.605 356.743,160.371 357.89,161.956 359.036,163.129 360.183,164.081 361.329,164.972 362.476,165.807 
 363.622,166.543 364.769,167.163 365.915,167.695 367.062,168.169 368.208,168.606 369.355,169.013 370.501,169.392 371.648,169.743 372.7

Fitting Done
typeof(data) = Array{Float64,1}
True loglikelihood      = -7131.114627750048
Estimated loglikelihood = -7129.9029596417495

DensityEstimationML.NeuralDensityEstimator{1}(Session(Ptr{Void} @0x00007fe57cff9b30), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [12]:
demo(GenerateDatasets.MagdonIsmailAndAtiya(), [32], 10_000)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
 
 1.5 
 
 
 2.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 42.4384,56.0184 44.8262,142.759 47.2141,146.957 49.6019,145.389 51.9897,136.65 54.3775,98.8426 56.7654,68.8084 59.1532,39.8899 61.541,19.7615 63.9289,12.6044 
 66.3167,11.811 68.7045,14.8051 71.0924,20.5263 73.4802,28.3668 75.868,37.8926 78.2558,48.7452 80.6437,60.5941 83.0315,73.1175 85.4193,85.9905 87.8072,98.8855 
 90.195,111.478 92.5828,123.459 94.9706,134.549 97.3585,144.506 99.7463,153.14 102.134,160.313 104.522,165.958 106.91,170.096 109.298,172.201 111.685,173.405 
 114.073,174.103 116.461,174.541 118.849,174.798 121.237,174.998 123.625,175.15 126.012,175.252 128.4,175.34 130.788,175.404 133.176,175.455 135.564,175.502 
 137.952,175.533 140.339,175.567 142.727,175.586 145.115,175.609 147.503,175.631 149.891,175.644 152.279,175.661 154.666,175.672 157.054,175.686 159.442,175.693 
 161.83,175.704 164.218,175.715 166.606,175.718 168.993,175.728 171.381,175.733 173.769,175.739 176.157,175.746 178.545,175.748 180.932,175.755 183.32,175.758 
 185.708,175.762 188.096,175.768 190.484,175.77 192.872,175.775 195.259,175.775 197.647,175.779 200.035,175.781 202.423,175.783 204.811,175.788 207.199,175.787 
 209.586,175.791 211.974,175.791 214.362,175.793 216.75,175.797 219.138,175.796 221.526,175.799 223.913,175.801 226.301,175.8 228.689,175.803 231.077,175.803 
 233.465,175.804 235.853,175.806 238.24,175.805 240.628,175.807 243.016,175.808 245.404,175.807 247.792,175.81 250.18,175.809 252.567,175.81 254.955,175.81 
 257.343,175.81 259.731,175.812 262.119,175.81 264.507,175.812 266.894,175.813 269.282,175.811 271.67,175.813 274.058,175.811 276.446,175.812 278.833,175.813 
 
 "/>
 
 
 
 
 ysmax 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
 
 800 
 
 
 850 
 
<polyline clip-path="url(#clip03)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 343.92,11.811 346.308,25.3631 348.696,25.4356 351.083,25.8898 353.471,36.6615 355.859,48.002 358.247,57.0515 360.635,65.0009 363.023,71.3261 365.41,76.5951 
 367.798,81.1382 370.186,85.1117 372.574,88.6186 374.962,91.7342 377.349,94.5187 379.737,97.0217 382.125,99.2858 384.513,101.347 386.901,103.238 389.289,104.986 
 391.676,106.615 394.064,108.146 396.452,109.596 398.84,110.979 401.228,112.305 403.616,113.582 406.003,114.816 408.391,116.01 410.779,117.161 413.167,118.283 
 415.555,119.385 417.943,120.476 420.33,121.565 422.718,122.654 425.106,123.746 427.494,124.841 429.882,125.937 432.27,127.038 434.657,128.138 437.045,129.237 
 439.433,130.333 441.821,131.425 444.209,132.511 446.597,133.591 448.984,134.663 451.372,135.727 453.76,136.781 456.148,137.826 458.536,138.86 460.924,139.882 
 463.311,140.894 465.699,141.894 468.087,142.882 470.475,143.857 472.863,144.817 475.25,145.771 477.638,146.708 480.026,147.633 482.414,148.545 484.802,149.444 
 487.19,150.331 489.577,151.204 491.965,152.064 494.353,152.912 496.741,153.747 499.129,154.57 501.517,155.379 503.904,156.177 506.292,156.962 508.68,157.734 
 511.068,158.495 513.456,159.241 515.844,159.98 518.231,160.705 520.619,161.418 523.007,162.119 525.395,162.807 527.783,163.488 530.171,164.156 532.558,164.809 
 534.946,165.458 537.334,166.093 539.722,166.717 542.11,167.33 544.498,167.932 546.885,168.526 549.273,169.109 551.661,169.682 554.049,170.244 556.437,170.795 
 558.824,171.341 561.212,171.874 563.6,172.398 565.988,172.913 568.376,173.419 570.764,173.915 573.151,174.403 575.539,174.882 577.927,175.352 580.315,175.813 
 
 "/>
 
 
 
 
 working_loss 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2500 
 
 
 5000 
 
 
 7500 
 
 
 -850 
 
 
 -800 
 
<polyline clip-path="url(#clip04)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 42.4384,375.813 44.8262,362.185 47.2141,362.104 49.6019,361.653 51.9897,350.90

Fitting Done
typeof(data) = Array{Float64,1}
True loglikelihood      = -784.7707758628787
Estimated loglikelihood = -795.5556815676257


DensityEstimationML.NeuralDensityEstimator{1}(Session(Ptr{Void} @0x00007fe57c4ee990), <Tensor Group:1 shape=unknown dtype=Complex{Float64}>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [ ]:
demo(Arcsine(1,4), [64,64], 20_000)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 5 
 
 
 10 
 
 
 
 
 
 
 ysmax 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 6000 
 
 
 6500 
 
 
 7000 
 
 
 
 
 
 
 working_loss 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 -7000 
 
 
 -6500 
 
 
 -6000 
 
 
 
 
 
 
 loglikelihood 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 1.5 
 
 
 2.0 
 
 
 
 
 
 
 ysmin

In [ ]:
dataset = GenerateDatasets.Likas1()
data = original_sample(dataset)
@show loglikelihood(dataset, data)
est = NeuralDensityEstimator([64], approximate_support(dataset))
condition!(est)
println("Conditioning Done")
fit!(est, data; epochs=10_000)
println("Fitting Done")

In [ ]:
gr =est.sess.graph
run(est.sess, exp(6*gr["W_2"].^2 + gr["b_2"]))

In [ ]:
collect(keys(est.sess.graph))